In [1]:
import requests
import os
import urllib3
import json

In [2]:
def get_text(file_path):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    headers = {
        'Authorization': 'Bearer YOUR_TOKEN_HERE',
    }

    try:
        with open(file_path, 'rb') as file:
            files = {
                'file': (os.path.basename(file_path), file, 'application/pdf')
            }

            response = requests.post('https://grupmedai-api-des.itcomb.cat/pdf/text', headers=headers, files=files, verify=False)

            if response.status_code == 200:
                response_data = response.json()
                if 'content' in response_data:
                    text = response_data['content']
                    return text
                else:
                    print("Error: 'content' key not found in the response.")
                    return response_data
            else:
                print(f"Error: Received status code {response.status_code}")
                print(f"Response: {response.json()}")
                return None

    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [3]:
get_text(r"C:\Users\Sophie\Tagging code\documents\HISTORIAL_MÈDIC.pdf")

'HISTORIAL MÈDIC \nHospital Clínic de Barcelona \nDepartament de Medicina Interna \n \nInformació del Pacient \n• \nNom complet: Marc Ferrer Puig \n• \nData de naixement: 5 d’abril de 1982 \n• \nEdat: 42 anys \n• \nGènere: Masculí \n• \nTelèfon: +34 678 123 456 \n• \nCorreu electrònic: marcferrer82@email.com \n• \nAdreça: Carrer Balmes, 123, 4t 2a, 08008 Barcelona \n• \nContacte d’emergència:  \no Nom: Laura Martí Soler (Esposa) \no Telèfon: +34 654 987 321 \n \nAntecedents Mèdics \n• \nMalalties cròniques: \no Hipertensió arterial diagnosticada el 2018 \no Diabetis tipus 2 diagnosticada el 2021 \no Asma lleu des de la infància \n• \nCirurgies prèvies: \no Apendicectomia (2005) \no Reparació d’hèrnia inguinal (2017) \n• \nAl·lèrgies conegudes: \no Penicil·lina (reacció anafilàctica) \no Fruits secs (urticària moderada) \n• \nHistorial familiar: \no Pare: Cardiopatia isquèmica (defunció als 65 anys) \no Mare: Diabetis tipus 2 \no Germà: Hipertensió arterial \n \nMedicació Actual \n• \nE

In [4]:
def upload_and_process(document_path, prompt=None):

    file_data = get_text(document_path)

    if prompt is None:
        prompt = f"""Label this document into one of the following categories: LEGAL, MEDICAL, EMAIL. 
        
        Return only the label. Example: LEGAL

        An email typically includes the sender and receiver addresses and some formalities such as a greeting and a closing. If the document is an EMAIL, but it has a medical or legal theme, tag as EMAIL nonetheless.
        
        Document information:
        {file_data}
        """

        prompt_legal = f"""This document is a LEGAL document. Add an appropriate sub-label:
        - ACTS (Record of previous hearing, Commission Minutes, Act of Conciliation, Act of preliminary diligences -- These documents formally record procedural steps, agreements, or outcomes within a legal or administrative process, often prepared by a judge, secretary, or official. They typically include the date, parties involved, the purpose or context of the meeting or action, and a summary of the decisions, declarations, or results.)
        - ACKNOWLEDGMENT OF RECEIPT (This document confirms that a person or entity has received a specific item, document, or notification, often noting the date and method of delivery. It typically includes the recipient’s name, a description of what was received, and their signature or formal confirmation.)
        - RESOLUTIONS OR CLOSURE (Archive/Dismissal, Decree, Administrative Resolution -- These documents formalize a decision that resolves, closes, or halts a legal or administrative proceeding, either partially or entirely. They typically include the reasoning behind the decision, the authority issuing it, and any legal consequences or next steps.)
        - NOTICES (Notice of Previous Hearing, Notice of Judgment -- These documents notify the parties of the date, time, and place set for a procedural event, such as a preliminary hearing or trial. They are issued by the court and typically include the case reference, type of hearing, and instructions for attendance or preparation.)
        - REPORTS (Attested, Medical Assesment Report, Claim Report, Resolution Letter, Lawyer's Report -- These documents present formal statements, evaluations, claims, or decisions submitted by parties, authorities, or legal professionals in the context of a legal or administrative procedure. They typically include the identity of the author, the facts or legal basis being addressed, and the purpose of the submission—whether to inform, accuse, defend, claim, or resolve.)
        - GUARANTEES OR DEPOSITS (Certificate of Insurance, Incident Communication, Bail -- These documents relate to the existence, activation, or guarantee of coverage or responsibility in the event of a loss or legal obligation. They typically specify the parties involved, the insured object or risk, and key terms such as coverage limits, incident details, or financial guarantees provided.)
        - ADMINISTRATIVE (Citation, Response to the Demand, Testimonial Statement, Demand, Complaint, Designate, Location, Doctor's File, Minute, Notification, Refusal, Requirement -- These documents are used to initiate, respond to, or support actions in legal or administrative proceedings, including summonses, claims, evidence, notices, or formal replies. They usually identify the parties involved, outline the purpose or action requested or taken, and include relevant procedural or factual details to move the process forward.)
        - PRELIMINARY PROCEEDINGS (Preliminary Diligences, Previous Diligences -- These documents record early investigative steps taken before or at the outset of a legal process, aimed at clarifying facts, preserving evidence, or identifying the parties involved. They typically include a description of the actions ordered or carried out, their legal basis, and the authority under which they are conducted.)
        - PAYMENTS OR COMPENSATIONS (Proof of Payment, Receipt, Collection, Cost Assessment -- These documents certify the payment, recovery, or assessment of financial amounts in a legal or contractual context. They typically state the amount involved, the parties concerned, the reason for the transaction, and may include signatures or official validation.)
        - APPEAL (Opposition Appeal, Appeal -- These documents are part of appellate procedures where a party either challenges a prior decision or responds in opposition to that challenge. They typically outline the legal grounds, relevant facts, and the relief sought or the reasons for upholding the original decision.)
        - POWER OF ATTORNEY (This document grants legal authority to a person (the attorney-in-fact) to act on behalf of another in legal or administrative matters. It typically specifies the scope of powers, the identities of the grantor and representative, and is signed before a notary or competent authority.)
        - SENTENCE (Appeal Sentence, Judgment of Cassation, Judgment of First Instance -- These are formal court decisions issued at various stages of the judicial process— first instance, appeal, and cassation or supreme court review. Each sentence includes the facts, legal reasoning, final ruling, and the court’s authority, reflecting the procedural level and scope of review.)
        - OTHER (Other documents, complementary documents)

        Use only categories that are in the list. Do NOT create new ones.

        Return only the sub-label. Example: REPORTS

        Document information:
        {file_data}
        """
        prompt_medical = f"""This document is a MEDICAL document. Add an appropriate sub-label:
        - REPORTS (Expanded Report, Forensic Report, External Expert Report, Internal Expert Report, Preliminary Report, Assessment of Bodily Harm -- These documents provide technical or professional assessments related to a medical, legal, or forensic matter, often used as evidence in investigations or proceedings. They typically include the expert’s credentials, methodology, findings, and conclusions—whether preliminary, expanded, internal, external, or specific.)
        - MEDICAL HISTORY (Petition, Authorization, Clinical History -- These documents are part of the management and access to a patient’s medical records, including the creation, request, or authorization to consult them. They typically contain identifying data, the purpose (e.g., treatment, legal, or administrative use), and the patient’s consent when required, especially in the authorization and petition documents.)
        - MEDICAL CERTIFICATE (This document is issued by a licensed healthcare professional to formally confirm a person’s health condition, diagnosis, treatment, or fitness for a specific activity. It typically includes the patient’s details, the medical findings, and the doctor's signature and credentials.)
        - INITIAL CLINICAL DIAGNOSIS (This document records the healthcare provider’s first assessment of a patient’s condition based on symptoms, medical history, and initial examination. It typically includes the suspected diagnosis, relevant findings, and a proposed plan for further testing or treatment.)
        - SEQUELAE STABILIZATION (This document confirms that the lasting effects (sequels) of an injury or illness have reached a stable state and are no longer expected to improve significantly. It typically includes a medical evaluation of the sequelae, their impact on function or quality of life, and the date of stabilization for legal or insurance purposes.)
        - OTHER

        Use only categories that are in the list. Do NOT create new ones.

        Return only the sub-label. Example: MEDICAL HISTORY

        Document information:
        {file_data}
        """

        prompt_email = f""" This document is an EMAIL. Add a sub-label: MEDICAL, LEGAL, OTHER.
        Return only the sub-label. Example: LEGAL

        Document information:
        {file_data}
        """

    try:
        destination_url = "https://grupmedai-api-des.itcomb.cat/llm/generate"
        headers = {
        'Authorization':  'Bearer YOUR_ACCESS_TOKEN',
        'Content-Type': 'application/json'
        }
        
        data = {
            "model": "llama3.1:8b",
            "prompt": prompt
        }

        response = requests.post(
            destination_url,
            headers=headers,
            data=json.dumps(data),
            verify=False
        )

        if response.status_code == 200:
            label = response.json().get('text', '')
        
        else:
            raise Exception(f"API request failed: {response.status_code}, {response.text}")
        
        sublabel = None
        
        if label == 'LEGAL':
            subprompt = prompt_legal
        elif label == 'MEDICAL':
            subprompt = prompt_medical
        elif label == 'EMAIL':
            subprompt = prompt_email
        
        data = {
                "model": "llama3.1:8b",
                "prompt": subprompt
            }

        response_sublabel = requests.post(
            destination_url,
            headers=headers,
            data=json.dumps(data),
            verify=False
        )

        if response_sublabel.status_code == 200:
            sublabel = response_sublabel.json().get('text', '')
            return {"label": label, "sublabel": sublabel}
            
        else:
            raise Exception(f"API request failed: {response_sublabel.status_code}, {response_sublabel.text}")
   
    except Exception as e:
        print(f"Error: {e}")
        return None

In [5]:
upload_and_process(r"C:\Users\Sophie\Tagging code\documents\HISTORIAL_MÈDIC.pdf")

{'label': 'MEDICAL', 'sublabel': 'MEDICAL HISTORY'}

In [6]:
upload_and_process(r"C:\Users\Sophie\Tagging code\documents\diagnostic_clinic_inicial.pdf")

{'label': 'MEDICAL', 'sublabel': 'MEDICAL HISTORY'}

In [7]:
upload_and_process(r"C:\Users\Sophie\Tagging code\documents\claim_medical_malpractice.pdf")

{'label': 'LEGAL', 'sublabel': 'ADMINISTRATIVE'}

In [8]:
upload_and_process(r"C:\Users\Sophie\Tagging code\documents\email_legal.pdf")

{'label': 'EMAIL', 'sublabel': 'LEGAL'}

In [9]:
upload_and_process(r"C:\Users\Sophie\Tagging code\documents\doctor_file.pdf")

{'label': 'MEDICAL', 'sublabel': 'MEDICAL HISTORY'}